In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time


driver = webdriver.firefox()
url = "https://www.booking.com/searchresults.fr.html?ss=Casablanca&checkin=2026-01-13&checkout=2026-01-14&group_adults=2&no_rooms=1&group_children=0&nflt=ht_id%3D204"
driver.get(url)

time.sleep(3)

soup = BeautifulSoup(driver.page_source, "html.parser")

data = []

hotels = soup.find_all("div", {"data-testid": "property-card"})

for hotel in hotels:
    # Nom
    title_tag = hotel.find("div", {"data-testid": "title"})
    title = title_tag.text.strip() if title_tag else "N/A"
    
    # Prix
    prix_tag = hotel.find("span", {"data-testid": "price-and-discounted-price"})
    prix = prix_tag.text.strip() if prix_tag else "N/A"
    
    # Localisation
    location_tag = hotel.find("span", {"data-testid": "address"})
    location = location_tag.text.strip() if location_tag else "N/A"
    
    # Note
    score_tag = hotel.find("div", {"data-testid": "review-score"})
    score = score_tag.text.strip() if score_tag else "N/A"
    
    # Nombre d’avis
    reviews_tag = hotel.find("div", {"data-testid": "review-count"})
    reviews = reviews_tag.text.strip() if reviews_tag else "N/A"
    
    # Lien
    link_tag = hotel.find("a", {"data-testid": "title-link"})
    link = "https://www.booking.com" + link_tag.get("href") if link_tag else "N/A"
    
    data.append({
        "Hotel": title,
        "Prix": prix,
        "Location": location,
        "Score": score,
        "Reviews": reviews,
        "Link": link
    })

driver.quit()

df = pd.DataFrame(data)
#df.to_csv("booking_casablanca_full.csv", index=False, encoding="utf-8")
print(df.head())
print("✅ Scraping terminé et CSV enrichi")



In [ ]:
import pandas as pd 

In [37]:
data = pd.read_csv("extract-data-2026-01-14.csv")
data.head()

,hotelName,city,price,location,starRating,userReviewScore,amenities
0,One Hotel Casablanca,Casablanca,US$67,"Sidi Belyout, 200 m du centre",5,7.4,"[{""value"":""Petit-déjeuner compris""},{""value"":""..."
1,Soho Boutique Casablanca,Casablanca,US$92,"Hay Hassani, 5.3 km du centre",5,7.6,"[{""value"":""Annulation gratuite""},{""value"":""Auc..."
2,Mövenpick Hotel Casablanca,Casablanca,US$106,"Sidi Belyout, 1.3 km du centre",5,8.0,"[{""value"":""Certificat de durabilité""},{""value""..."
3,Barceló Anfa Casablanca,Casablanca,US$154,"Sidi Belyout, 1.2 km du centre",5,8.4,"[{""value"":""Wi-Fi gratuite""},{""value"":""Climatis..."
4,Art Palace Suites & Spa,Casablanca,US$152,"Sidi Belyout (Gauthier), 1.7 km du centre",5,8.7,"[{""value"":""Wi-Fi gratuite""},{""value"":""Climatis..."


In [ ]:
data['city'].unique()

In [38]:
data[["Quartier", "Distance"]] = data["location"].str.split(",", n=1, expand=True)



In [ ]:
data.head()



In [ ]:
data.head()

In [39]:
import json

data["amenities_clean"] = data["amenities"].apply(
    lambda x: [d.get("value") for d in json.loads(x)] if isinstance(x, str) else []
)
data["amenities_text"] = data["amenities_clean"].apply(lambda lst: ", ".join(lst))

In [40]:
data.head()


,hotelName,city,price,location,starRating,userReviewScore,amenities,Quartier,Distance,amenities_clean,amenities_text
0,One Hotel Casablanca,Casablanca,US$67,"Sidi Belyout, 200 m du centre",5,7.4,"[{""value"":""Petit-déjeuner compris""},{""value"":""...",Sidi Belyout,200 m du centre,"[Petit-déjeuner compris, Annulation gratuite, ...","Petit-déjeuner compris, Annulation gratuite, W..."
1,Soho Boutique Casablanca,Casablanca,US$92,"Hay Hassani, 5.3 km du centre",5,7.6,"[{""value"":""Annulation gratuite""},{""value"":""Auc...",Hay Hassani,5.3 km du centre,"[Annulation gratuite, Aucun prépaiement requis...","Annulation gratuite, Aucun prépaiement requis,..."
2,Mövenpick Hotel Casablanca,Casablanca,US$106,"Sidi Belyout, 1.3 km du centre",5,8.0,"[{""value"":""Certificat de durabilité""},{""value""...",Sidi Belyout,1.3 km du centre,"[Certificat de durabilité, Wi-Fi gratuite, Cli...","Certificat de durabilité, Wi-Fi gratuite, Clim..."
3,Barceló Anfa Casablanca,Casablanca,US$154,"Sidi Belyout, 1.2 km du centre",5,8.4,"[{""value"":""Wi-Fi gratuite""},{""value"":""Climatis...",Sidi Belyout,1.2 km du centre,"[Wi-Fi gratuite, Climatisation, Parking, Servi...","Wi-Fi gratuite, Climatisation, Parking, Servic..."
4,Art Palace Suites & Spa,Casablanca,US$152,"Sidi Belyout (Gauthier), 1.7 km du centre",5,8.7,"[{""value"":""Wi-Fi gratuite""},{""value"":""Climatis...",Sidi Belyout (Gauthier),1.7 km du centre,"[Wi-Fi gratuite, Climatisation, Parking, Servi...","Wi-Fi gratuite, Climatisation, Parking, Servic..."


In [ ]:
data.columns

Index(['hotelName', 'city', 'price', 'location', 'starRating',
       'userReviewScore', 'amenities', 'Quartier', 'Distance',
       'amenities_clean', 'amenities_text'],
      dtype='object')

In [ ]:
df_clean = data.drop(columns=["amenities", "amenities_clean"])

df_clean.head()


,hotelName,city,price,location,starRating,userReviewScore,Quartier,Distance,amenities_text
0,One Hotel Casablanca,Casablanca,US$67,"Sidi Belyout, 200 m du centre",5,7.4,Sidi Belyout,200 m du centre,"Petit-déjeuner compris, Annulation gratuite, W..."
1,Soho Boutique Casablanca,Casablanca,US$92,"Hay Hassani, 5.3 km du centre",5,7.6,Hay Hassani,5.3 km du centre,"Annulation gratuite, Aucun prépaiement requis,..."
2,Mövenpick Hotel Casablanca,Casablanca,US$106,"Sidi Belyout, 1.3 km du centre",5,8.0,Sidi Belyout,1.3 km du centre,"Certificat de durabilité, Wi-Fi gratuite, Clim..."
3,Barceló Anfa Casablanca,Casablanca,US$154,"Sidi Belyout, 1.2 km du centre",5,8.4,Sidi Belyout,1.2 km du centre,"Wi-Fi gratuite, Climatisation, Parking, Servic..."
4,Art Palace Suites & Spa,Casablanca,US$152,"Sidi Belyout (Gauthier), 1.7 km du centre",5,8.7,Sidi Belyout (Gauthier),1.7 km du centre,"Wi-Fi gratuite, Climatisation, Parking, Servic..."


In [55]:
data['city'].unique()

array(['Casablanca', 'Rabat', 'Tangier', 'Marrakech'], dtype=object)

In [57]:
data.shape 

(89, 11)

In [61]:
dff = pd.read_csv("extract-data-2026-01-15.csv")
dff.head()


,city,city_citation,provider,provider_citation,vehicle_model,vehicle_model_citation,price_mad,price_mad_citation,currency,currency_citation,pickup_date,pickup_date_citation,dropoff_date,dropoff_date_citation,booking_com_url,booking_com_url_citation
0,Marrakech,https://cars.booking.com/package/deal/eyJkcml2...,Surprice,https://cars.booking.com/package/deal/eyJkcml2...,Hyundai i10,https://cars.booking.com/package/deal/eyJkcml2...,259,https://cars.booking.com/package/deal/eyJkcml2...,MAD,https://cars.booking.com/package/deal/eyJkcml2...,2026-01-18,https://cars.booking.com/package/deal/eyJkcml2...,2026-01-19,https://cars.booking.com/package/deal/eyJkcml2...,https://cars.booking.com/package/deal/eyJkcml2...,https://cars.booking.com/package/deal/eyJkcml2...
1,Marrakech,https://cars.booking.com/package/deal/eyJkcml2...,Surprice,https://cars.booking.com/package/deal/eyJkcml2...,Dacia Sandero,https://cars.booking.com/package/deal/eyJkcml2...,270,https://cars.booking.com/package/deal/eyJkcml2...,MAD,https://cars.booking.com/package/deal/eyJkcml2...,2026-01-18,https://cars.booking.com/package/deal/eyJkcml2...,2026-01-19,https://cars.booking.com/package/deal/eyJkcml2...,https://cars.booking.com/package/deal/eyJkcml2...,https://cars.booking.com/package/deal/eyJkcml2...
2,Marrakech,https://cars.booking.com/package/deal/eyJkcml2...,U-Save,https://cars.booking.com/package/deal/eyJkcml2...,Hyundai I10,https://cars.booking.com/package/deal/eyJkcml2...,277,https://cars.booking.com/package/deal/eyJkcml2...,MAD,https://cars.booking.com/package/deal/eyJkcml2...,2026-01-18,https://cars.booking.com/package/deal/eyJkcml2...,2026-01-19,https://cars.booking.com/package/deal/eyJkcml2...,https://cars.booking.com/package/deal/eyJkcml2...,https://cars.booking.com/package/deal/eyJkcml2...
3,Marrakech,https://cars.booking.com/package/deal/eyJkcml2...,U-Save,https://cars.booking.com/package/deal/eyJkcml2...,Dacia Sandero,https://cars.booking.com/package/deal/eyJkcml2...,285,https://cars.booking.com/package/deal/eyJkcml2...,MAD,https://cars.booking.com/package/deal/eyJkcml2...,2026-01-18,https://cars.booking.com/package/deal/eyJkcml2...,2026-01-19,https://cars.booking.com/package/deal/eyJkcml2...,https://cars.booking.com/package/deal/eyJkcml2...,https://cars.booking.com/package/deal/eyJkcml2...
4,Marrakech,https://cars.booking.com/package/deal/eyJkcml2...,Surprice,https://cars.booking.com/package/deal/eyJkcml2...,Kia Picanto,https://cars.booking.com/package/deal/eyJkcml2...,302,https://cars.booking.com/package/deal/eyJkcml2...,MAD,https://cars.booking.com/package/deal/eyJkcml2...,2026-01-18,https://cars.booking.com/package/deal/eyJkcml2...,2026-01-19,https://cars.booking.com/package/deal/eyJkcml2...,https://cars.booking.com/package/deal/eyJkcml2...,https://cars.booking.com/package/deal/eyJkcml2...
